Import des bibliothèque necessaires

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.stem.snowball import SnowballStemmer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import TruncatedSVD, NMF
#nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...


True

Class Data pour tous les pré-traitement, vectorisation et réduction de la matrice.

In [ ]:
class Data:
    def __init__(self, limit=5000):
        self.data_brut = pd.read_json("train.jsonl", lines=True)[["texte_annonce", "cal_réponse_signalement"]]
        self.data = self.data_brut.iloc[:limit]

    def get_data(self):
        return self.data

    def rien(self, x):
        pass

    def racinisation(self, text_column):
        stemmer = SnowballStemmer("french")
        self.data.loc[:, text_column] = self.data[text_column].apply(lambda x: ' '.join([stemmer.stem(word) for word in str(x).split()]))

    def lemmatisation(self, text_column):
        lemmatizer = WordNetLemmatizer()
        self.data.loc[:, text_column] = self.data[text_column].apply(lambda x: ' '.join([lemmatizer.lemmatize(word) for word in str(x).split()]))

    def vectorisation_simple(self, text_column):
        vectorizer = CountVectorizer()
        vect_data = vectorizer.fit_transform(self.data[text_column])
        vect_df = pd.DataFrame(vect_data.toarray(), columns=vectorizer.get_feature_names_out())
        self.data = pd.concat([vect_df, self.data.iloc[:, -1]], axis=1)

    def vectorisation_ponderee(self, text_column):
        vectorizer = TfidfVectorizer()
        vect_data = vectorizer.fit_transform(self.data[text_column])
        vect_df = pd.DataFrame(vect_data.toarray(), columns=vectorizer.get_feature_names_out())
        self.data = pd.concat([vect_df, self.data.iloc[:, -1]], axis=1)

    def reduction_svd(self, n_components=100):
        svd = TruncatedSVD(n_components=n_components)
        svd_result = svd.fit_transform(self.data.iloc[:, :-1])
        self.data = pd.concat([pd.DataFrame(svd_result), self.data.iloc[:, -1]], axis=1)

    def reduction_nmf(self, n_components=100):
        nmf = NMF(n_components=n_components)
        nmf_result = nmf.fit_transform(self.data.iloc[:, :-1])
        self.data = pd.concat([pd.DataFrame(nmf_result), self.data.iloc[:, -1]], axis=1)

    def process_and_export(self, text_column="texte_annonce", n_components=100):
        # Définir toutes les combinaisons possibles
        preprocessing_methods = [self.rien, self.racinisation, self.lemmatisation]
        vectorisation_methods = [self.vectorisation_simple, self.vectorisation_ponderee]
        reduction_methods = [self.reduction_svd, self.reduction_nmf]

        combinations = product(preprocessing_methods, vectorisation_methods, reduction_methods)

        # Boucler sur toutes les combinaisons
        for i, (preprocess, vectorize, reduce) in enumerate(combinations):
            # Réinitialiser les données à l'état brut
            self.data = self.data_brut.copy()

            # Appliquer les méthodes
            preprocess(text_column)
            vectorize(text_column)
            reduce(n_components)

            # Exporter le résultat
            output_file = f"output_combination_{i+1}.jsonl"
            self.data.to_json(output_file, orient="records", lines=True)
            print(f"Exporté : {output_file}")



Class global des classifieurs permettant de faire de l'héritage

In [ ]:
class Classifieur:
    def __init__(self, data):
        # Chargement et découpage des données
        self.chargement(data)
        self.decoupage()

    def chargement(self, data):
        # Chargement des données et séparation X, y
        self.data = data
        self.X = data.iloc[:, :-1]  # Toutes les colonnes sauf la dernière (X)
        self.y = data.iloc[:, -1]   # Dernière colonne (y)

    def decoupage(self):
        # Découpage en jeu d'entrainement et test
        self.X_train, self.X_test, self.Y_train, self.Y_test = train_test_split(self.X, self.y, test_size=0.6, random_state=0)

    def entrainement(self):
        pass

    def rien(self):
        pass

    def taux_reussite(self):
        # Calcul du taux de réussite
        y_pred = self.classifier.predict(self.X_test)
        accuracy = accuracy_score(self.Y_test, y_pred)
        return f"{accuracy:.3f}"

    def f1_score(self):
        # Calcul du F1-score
        from sklearn.metrics import f1_score
        y_pred = self.classifier.predict(self.X_test)
        f1 = f1_score(self.Y_test, y_pred, average='weighted')
        return f"{f1:.3f}"

    def liste_reussite(self):
        pass

    def graph_train_vs_test(self, titre_graph, axe_x, nom_x, list_reussite_train, list_reussite_test):
        # Affichage d'un graphique des taux de réussite pour l'entrainement et la validation
        plt.figure(figsize=(10, 6))
        plt.plot(axe_x, list_reussite_train, label=f"% réussite entrainement", marker='o')
        plt.plot(axe_x, list_reussite_test, label=f"% réussite validation", marker='o')
        plt.xlabel(nom_x)
        plt.ylabel("Taux de réussite")
        plt.title(titre_graph)
        plt.legend()
        plt.show()

    def graph_f1_train_vs_test(self, titre_graph, axe_x, nom_x, list_f1_train, list_f1_test):
        # Affichage d'un graphique des F1-scores pour l'entrainement et la validation
        plt.figure(figsize=(10, 6))
        plt.plot(axe_x, list_f1_train, label=f"F1-score entraînement", marker='o')
        plt.plot(axe_x, list_f1_test, label=f"F1-score validation", marker='o')
        plt.xlabel(nom_x)
        plt.ylabel("F1-score")
        plt.title(titre_graph)
        plt.legend()
        plt.show()



In [ ]:
#rajouter bernouillli et multit-nomiale

from sklearn.naive_bayes import GaussianNB,MultinomialNB,BernoulliNB


class BayesienNaif(Classifieur):
    def __init__(self, data, mode=None, alpha = None, fit_prior = None, var_smoothing = None, binarize = None):
        super().__init__(data)
        self.mode = mode

        if self.mode == 'multinomial':
            self.classifier = MultinomialNB()
            if alpha : self.classifier.alpha = alpha
            if fit_prior : self.classifier.fit_prior = fit_prior
        elif self.mode == 'bernoulli':
            self.classifier = BernoulliNB()
            if alpha : self.classifier.alpha = alpha
            if fit_prior : self.classifier.fit_prior = fit_prior
            if binarize : self.classifier.binarize = binarize
        else:  # Par défaut, on utilise le modèle gaussien
            self.classifier = GaussianNB()
            if var_smoothing : self.classifier.var_smoothing = var_smoothing

    def entrainement(self):
        # Entraînement du modèle avec les données
        self.classifier.fit(self.X_train, self.Y_train)


In [ ]:
#Rajouter les régularisation

from sklearn.linear_model import LogisticRegression

class RegressionLogistique(Classifieur):
    def __init__(self, data, penalty = None, C = None, solver = None, max_iter = None, l1_ratio = None, random_state = 0):
        super().__init__(data)
        self.classifier = LogisticRegression()
        if penalty : self.classifier.penalty = penalty
        if C : self.classifier.C = C
        if solver : self.classifier.solver = solver
        if max_iter : self.classifier.max_iter = max_iter
        if l1_ratio : self.classifier.l1_ratio
        if random_state : self.classifier.random_state = random_state

    def entrainement(self):
        self.classifier.fit(self.X_train, self.Y_train)


In [ ]:
from sklearn.neighbors import KNeighborsClassifier

class KPlusProchesVoisins(Classifieur):
    def __init__(self, data, n_neighbors = None, weights = None, algorithm = None, p = None):
        super().__init__(data)

        self.classifier = KNeighborsClassifier()
        if n_neighbors : self.classifier.n_neighbors = n_neighbors
        if weights : self.classifier.weights = weights
        if algorithm : self.classifier.algorithm = algorithm
        if p : self.classifier.p = p

    def entrainement(self, n_neighbors):
        self.classifier.fit(self.X_train, self.Y_train)


In [ ]:
from sklearn.tree import DecisionTreeClassifier

class ArbreDeDecision(Classifieur):
    def __init__(self, data, criterion = None, max_depth = None, min_samples_split = None, min_samples_leaf = None, max_features = None, ccp_alpha = None, random_state = 0):
        super().__init__(data)
        self.classifier = DecisionTreeClassifier()
        if criterion : self.classifier.criterion = criterion
        if max_depth : self.classifier.max_depth = max_depth
        if min_samples_split : self.classifier.min_samples_split = min_samples_split
        if min_samples_leaf : self.classifier.min_samples_leaf = min_samples_leaf
        if max_features : self.classifier.max_features = max_features
        if ccp_alpha : self.classifier.ccp_alpha = ccp_alpha
        if random_state : self.classifier.random_state = random_state

    def entrainement(self):
        self.classifier.fit(self.X_train, self.Y_train)


In [ ]:
from sklearn.ensemble import RandomForestClassifier

class ForetAleatoire(Classifieur):
    def __init__(self, data, n_estimators = None, criterion = None, max_depth = None, min_samples_split = None, min_samples_leaf = None, max_features = None, bootstrap = None, ccp_alpha = None, random_state = 0 ):
        super().__init__(data)
        self.classifier = RandomForestClassifier()
        if n_estimators : self.classifier.n_estimators = n_estimators
        if criterion : self.classifier.criterion = criterion
        if max_depth : self.classifier.max_depth = max_depth
        if min_samples_split : self.classifier.min_samples_split = min_samples_split
        if min_samples_leaf : self.classifier.min_samples_leaf = min_samples_leaf
        if max_features : self.classifier.max_features = max_features
        if bootstrap : self.classifier.bootstrap = bootstrap
        if ccp_alpha : self.classifier.ccp_alpha = ccp_alpha
        if random_state : self.classifier.random_state = random_state

    def entrainement(self, n_estimators=100):
        self.classifier.fit(self.X_train, self.Y_train)


Fonction ultra débile qui teste tous les classifieurs, pré traitements, vectorisation et réduction afin d'avoir le meilleurs taux de réussite.

In [ ]:
def possibilite():
    results = []  # Liste pour stocker les résultats

    for x in ["lemmatisation", "racinisation","rien"]:
        for y in ["vectorisation_simple", "vectorisation_ponderee"]:
            data = Data()
            getattr(data, x)("texte_annonce")
            getattr(data, y)("texte_annonce")

            if y == "vectorisation_ponderee":
                data.reduction_nmf()
                z = "nmf"
            else:
                data.reduction_svd()
                z = "svd"

            # Parcours des classifieurs
            for i in [BayesienNaif, RegressionLogistique, KPlusProchesVoisins, ArbreDeDecision, ForetAleatoire]:
                classifieur = i(data.get_data())
                classifieur.entrainement()
                taux_reussite = classifieur.taux_reussite()
                f1_score = classifieur.f1_score()

                # Ajout des résultats à la liste sous forme de tuple
                results.append((x, y, z, i.__name__, taux_reussite,f1_score))

    # Tri des résultats par taux de réussite (du plus élevé au plus faible)
    results_sorted = sorted(results, key=lambda x: x[4], reverse=True)

    # Affichage des résultats triés
    for res in results_sorted:
        print(f"{res[0]} - {res[1]} - {res[2]} - {res[3]} : {res[4]}")

possibilite()

Résultat lancé en amont (tps: 16 min)

- lemmatisation - vectorisation_ponderee - nmf - ForetAleatoire : 0.741
- racinisation - vectorisation_ponderee - nmf - ForetAleatoire : 0.738
- rien - vectorisation_ponderee - nmf - ForetAleatoire : 0.737
- lemmatisation - vectorisation_ponderee - nmf - RegressionLogistique : 0.734
- racinisation - vectorisation_simple - svd - ForetAleatoire : 0.734
- racinisation - vectorisation_ponderee - nmf - RegressionLogistique : 0.734
- rien - vectorisation_ponderee - nmf - RegressionLogistique : 0.734
- lemmatisation - vectorisation_simple - svd - ForetAleatoire : 0.733
- rien - vectorisation_simple - svd - ForetAleatoire : 0.732
- lemmatisation - vectorisation_simple - svd - RegressionLogistique : 0.725
- racinisation - vectorisation_simple - svd - RegressionLogistique : 0.723
- rien - vectorisation_simple - svd - RegressionLogistique : 0.723
- rien - vectorisation_ponderee - nmf - KPlusProchesVoisins : 0.667
- lemmatisation - vectorisation_ponderee - nmf - KPlusProchesVoisins : 0.658
- racinisation - vectorisation_ponderee - nmf - KPlusProchesVoisins : 0.657
- racinisation - vectorisation_simple - svd - KPlusProchesVoisins : 0.651
- lemmatisation - vectorisation_simple - svd - KPlusProchesVoisins : 0.648
- rien - vectorisation_simple - svd - KPlusProchesVoisins : 0.644
- lemmatisation - vectorisation_ponderee - nmf - ArbreDeDecision : 0.643
- racinisation - vectorisation_ponderee - nmf - ArbreDeDecision : 0.640
- rien - vectorisation_ponderee - nmf - ArbreDeDecision : 0.637
- lemmatisation - vectorisation_simple - svd - ArbreDeDecision : 0.623
- rien - vectorisation_simple - svd - ArbreDeDecision : 0.617
- racinisation - vectorisation_simple - svd - ArbreDeDecision : 0.586
- rien - vectorisation_ponderee - nmf - BayesienNaif : 0.494
- lemmatisation - vectorisation_ponderee - nmf - BayesienNaif : 0.491
- racinisation - vectorisation_ponderee - nmf - BayesienNaif : 0.483
- lemmatisation - vectorisation_simple - svd - BayesienNaif : 0.329
- rien - vectorisation_simple - svd - BayesienNaif : 0.328
- racinisation - vectorisation_simple - svd - BayesienNaif : 0.326


Exemple d'utilisation en temps normal pour l'optimisation

In [ ]:
data = Data()
data.lemmatisation("texte_annonce")
data.vectorisation_ponderee("texte_annonce")
data.reduction_nmf()
data = data.get_data()

foret = ForetAleatoire(data)
foret.entrainement()
print(foret.taux_reussite())
print(foret.f1_score())

/usr/local/lib/python3.10/dist-packages/sklearn/decomposition/_nmf.py:1759: ConvergenceWarning: Maximum number of iterations 200 reached. Increase it to improve convergence.
  warnings.warn(


0.739


Optimisation de la foret

In [ ]:
#Donne les meilleurs hyperparametre de la foret
param_grid = {
    'n_estimators': [100, 200, 500],
    'max_depth': [10, 20, None],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 5],
    'max_features': ['sqrt', 'log2', None]
}

data = Data()
data.lemmatisation("texte_annonce")
data.vectorisation_ponderee("texte_annonce")
data.reduction_nmf()
data = data.get_data()

foret = ForetAleatoire(data)
foret.recherche_hyperparametres(param_grid)

NameError: name 'Data' is not defined

In [ ]:
#Prédiction avec les meilleurs hyperparametre
data = Data()
data.lemmatisation("texte_annonce")
data.vectorisation_ponderee("texte_annonce")
data.reduction_nmf()
data = data.get_data()

foret = ForetAleatoire(data)
foret.entrainement(max_depth=20, max_features="sqrt", min_samples_leaf=2, min_samples_split=5, n_estimators=200)
print("Taux de réussite :", foret.taux_reussite())
print("F1 score :", foret.f1_score())

